# 📄 Documentação passo a passo — View Temporária `BronzeYellowMes03`

Este notebook prepara e filtra dados das viagens **Yellow Taxi (Marco/2023)**,
criando uma tabela com métricas derivadas tratadas e regras de consistência temporal.

---

## 🎯 Objetivo
Criar a **temp view** `BronzeYellowMes03` a partir de `RawYellowMes03`,
aplicando:
- derivação de campos de data/hora e duração,
- regras de qualidade de dados,
- consistência entre início e fim da viagem.

---


In [0]:
%sql

USE CATALOG ifood_case; -- indica o catálogo selecionado
USE SCHEMA  nytaxi;     -- indica o schema selecionado

-- Cria (ou substitui) a view temporária com as etapas de limpeza e validação
CREATE OR REPLACE TEMP VIEW BronzeYellowMes03STG AS
-- =========================================
-- 1) Limpeza e derivação de colunas base
--    (CTE: EspecificationDataClean)
-- =========================================
WITH EspecificationDataClean AS
(
  SELECT 
    *                                                       -- mantém todas as colunas originais do bronze
    -- 🔹 Datas derivadas (formato ISO-friendly)
    , DATE_FORMAT(tpep_pickup_datetime,  'yyyy-MM-dd') AS Data_Inicio_Viagem
    , DATE_FORMAT(tpep_dropoff_datetime, 'yyyy-MM-dd') AS Data_Fim_Viagem
    -- 🔹 Componentes de dia (número do dia no mês)
    , DAY(tpep_dropoff_datetime)                       AS Dia_Fim_Viagem   
    , DAY(tpep_pickup_datetime)                        AS Dia_Inicio_Viagem 
    -- 🔹 Duração da viagem em minutos (2 casas decimais)
    , ROUND(
        (UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) / 60
      , 2
      )                                                AS Duracao_Viagem
    -- 🔹 Hora do dia (0–23) — somente é considerada a HORA CHEIA, sem minutos
    , DATE_FORMAT(tpep_dropoff_datetime, 'HH')         AS Hora_Fim_Viagem
    , DATE_FORMAT(tpep_pickup_datetime,  'HH')         AS Hora_Inicio_Viagem
    -- 🔹 Componentes de mês (1–12)
    , MONTH(tpep_dropoff_datetime)                     AS Mes_Fim_Viagem
    , MONTH(tpep_pickup_datetime)                      AS Mes_Inicio_Viagem        
-- Tabela com dados brutos (RAW Table) do mês de Marco /2023
  FROM RawYellowMes03
  -- -----------------------------------------
  -- Filtros iniciais de integridade de dados
  -- -----------------------------------------
  WHERE
    improvement_surcharge > 0                          -- taxa de melhoria deve ser positiva e obrigatória
    AND mta_tax           > 0                          -- taxa MTA deve ser positiva e obrigatória
    AND passenger_count   > 0                          -- ao menos 1 passageiro
    AND (payment_type BETWEEN 0 AND 6)                 -- tipos de pagamento válidos
    AND (PULocationID <> DOLocationID)                 -- embarque ≠ desembarque
    AND (store_and_fwd_flag   = 'N' 
        OR store_and_fwd_flag = 'Y')                   -- flag válida
    AND total_amount > 0                               -- valor do pagamento total positivo
    -- verificação se o total pago é igual a soma das taxas
    AND (ROUND(total_amount,2) 
        = ROUND(
          (fare_amount
          + extra 
          + mta_tax
          + tip_amount
          + tolls_amount
          + improvement_surcharge
          + airport_fee
          + congestion_surcharge), 2))
    AND trip_distance > 0                              -- distância positiva
    AND vendorid IN (1, 2, 6, 7)                       -- vendors permitidos
),
-- =========================================
-- 2) Regras de consistência temporal
--    (CTE: DateTimeDataClean)
-- =========================================
DateTimeDataClean AS
(
  SELECT 
    *                                                  -- mantém os dados do filtro EspecificationDataClean
  FROM 
    EspecificationDataClean 
  WHERE
    Duracao_Viagem          > 0                        -- duração precisa ser maior do que zero indicando que houve deslocamento
    AND Mes_Fim_Viagem      = 3                        -- mês fim: Marco
    AND Mes_Inicio_Viagem   = 3                        -- mês início: Marco
    AND Hora_Inicio_Viagem  BETWEEN '00' AND '23'      -- valida intervalo de horas do inicio da viagem
    AND Hora_Fim_Viagem     BETWEEN '00' AND '23'      -- valida intervalo de horas do fim da viagem
    AND Dia_Inicio_Viagem   BETWEEN 1 AND 31           -- valida intervalo de dias do mês do inicio da viagem
    AND Dia_Fim_Viagem      BETWEEN 1 AND 31           -- valida intervalo de dias do mês do fim da viagem
    -- -----------------------------------------
    -- Consistência entre datas/horas de início e fim
    -- -----------------------------------------
    AND (
         -- (A) Mesmo mês e dia inicial <= dia final
         ((Dia_Inicio_Viagem <= Dia_Fim_Viagem) AND (Mes_Fim_Viagem = Mes_Inicio_Viagem))
         -- (B) Virada de mês: dia inicial > dia final e meses diferentes
         OR ((Dia_Inicio_Viagem > Dia_Fim_Viagem) AND (Mes_Fim_Viagem <> Mes_Inicio_Viagem))
         -- (C) Horário cruza para o mesmo dia (hora início > hora fim),
         --     mas dias coerentes (início < fim) e mesmo mês
         OR ((Hora_Inicio_Viagem > Hora_Fim_Viagem)  
             AND ((Dia_Inicio_Viagem < Dia_Fim_Viagem) AND (Mes_Fim_Viagem = Mes_Inicio_Viagem)))
         -- (D) Mesma data e hora início < hora fim
         OR ((Hora_Inicio_Viagem < Hora_Fim_Viagem) 
             AND ((Dia_Inicio_Viagem = Dia_Fim_Viagem) AND (Mes_Fim_Viagem = Mes_Inicio_Viagem)))
        )
)
-- =========================================
-- 3) Seleção final: colunas relevantes
-- =========================================
SELECT 
  -- 💰 Componentes financeiros
  airport_fee,
  congestion_surcharge,
  extra,
  fare_amount,
  improvement_surcharge,
  mta_tax,
  tip_amount,
  tolls_amount,
  total_amount,
  -- 📅 Datas e horas (originais e derivadas)
  tpep_pickup_datetime,
  tpep_dropoff_datetime,
  Data_Inicio_Viagem,
  Data_Fim_Viagem,
  Hora_Inicio_Viagem,
  Hora_Fim_Viagem,
  Dia_Inicio_Viagem,
  Dia_Fim_Viagem,
  Mes_Inicio_Viagem,
  Mes_Fim_Viagem,
  -- 📍 Geografia e metadados
  PULocationID,
  DOLocationID,
  trip_distance,
  passenger_count,
  payment_type,
  store_and_fwd_flag,
  vendorId
FROM
  DateTimeDataClean;

---


# 🧪 Validação de Qualidade — `BronzeYellowMes03STG`
Este notebook gera um **campo agregado de alertas** (`alerta_validacao`)
com base em regras de consistência de negócio e integridade do dado (valores nulos, intervalos inválidos,
soma de taxas, etc.). Ao final, exibe apenas os registros com algum problema.


In [0]:
%python
# Importando funcionalidades 
from pyspark.sql.functions import col, when, lit, concat, unix_timestamp, round

# Tabela preparada em estágio "bronze"
df = spark.table("BronzeYellowMes03STG")

# Conjunto de vendors aceitos (regra de negócio)
vendors_validos = [1, 2, 6, 7]

# -------------------------------------------------------------------
# 1) Cálculo de métricas derivadas + construção do campo de alertas
# -------------------------------------------------------------------
# - Duracao_Viagem: diferença (dropoff - pickup) em minutos, arredondada para 2 casas
# - alerta_validacao: CONCAT de mensagens (strings) disparadas pelas regras
# - cada WHEN gera a mensagem da regra quando a condição é verdadeira; caso contrário, devolve "".
#   O uso de strings vazias permite manter compatível com versões antigas do Spark.
# -------------------------------------------------------------------
df_validacao = (
    df.withColumn(
        "Duracao_Viagem",
        round((unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 60, 2)
    ).withColumn(
        "alerta_validacao",
        concat(
            # -----------------------------
            # Regras de datas/dias/meses
            # -----------------------------
            when(
                col("Dia_Fim_Viagem").isNull() | (col("Dia_Fim_Viagem") < 1) | (col("Dia_Fim_Viagem") > 31),
                lit("Dia_Fim_Viagem fora do intervalo")
            ).otherwise(lit("")),
            when(
                col("Dia_Inicio_Viagem").isNull() | (col("Dia_Inicio_Viagem") < 1) | (col("Dia_Inicio_Viagem") > 31),
                lit("Dia_Inicio_Viagem fora do intervalo")
            ).otherwise(lit("")),
            # Duração deve ser positiva
            when(
                col("Duracao_Viagem") <= 0,
                lit("Duracao_Viagem inválida")
            ).otherwise(lit("")),
            # -----------------------------
            # Regras de hora (formato texto)
            # -----------------------------
            when(
                col("Hora_Fim_Viagem").isNull() | (col("Hora_Fim_Viagem") < '00') | (col("Hora_Fim_Viagem") > '23'),
                lit("Hora_Fim_Viagem fora do intervalo")
            ).otherwise(lit("")),
            when(
                col("Hora_Inicio_Viagem").isNull() | (col("Hora_Inicio_Viagem") < '00') | (col("Hora_Inicio_Viagem") > '23'),
                lit("Hora_Inicio_Viagem fora do intervalo")
            ).otherwise(lit("")),
            # -----------------------------
            # Regras de mês (escopo do dataset)
            # -----------------------------
            when(
                col("Mes_Fim_Viagem").isNull() | (col("Mes_Fim_Viagem") < 1) | (col("Mes_Fim_Viagem") > 5),
                lit("Mes_Fim_Viagem fora do intervalo")
            ).otherwise(lit("")),
            when(
                col("Mes_Inicio_Viagem").isNull() | (col("Mes_Inicio_Viagem") < 1) | (col("Mes_Inicio_Viagem") > 5),
                lit("Mes_Inicio_Viagem fora do intervalo")
            ).otherwise(lit("")),
            # -----------------------------
            # Regras de geografia (zonas)
            # -----------------------------
            when(
                col("PULocationID").isNull() | col("DOLocationID").isNull() | (col("PULocationID") == col("DOLocationID")),
                lit("PULocationID e DOLocationID não podem ser iguais")
            ).otherwise(lit("")),
            # -----------------------------
            # Regras financeiras (taxas > 0)
            # -----------------------------
            when(
                col("improvement_surcharge").isNull() | (col("improvement_surcharge") <= 0),
                lit("improvement_surcharge inválido")
            ).otherwise(lit("")),
            when(
                col("mta_tax").isNull() | (col("mta_tax") <= 0),
                lit("mta_tax inválido")
            ).otherwise(lit("")),
            # -----------------------------
            # Outras consistências de negócio
            # -----------------------------
            when(
                col("passenger_count").isNull() | (col("passenger_count") <= 0),
                lit("passenger_count inválido")
            ).otherwise(lit("")),
            when(
                col("payment_type").isNull() | (col("payment_type") < 0) | (col("payment_type") > 6),
                lit("payment_type inválido")
            ).otherwise(lit("")),
            when(
                col("store_and_fwd_flag").isNull() | (~col("store_and_fwd_flag").isin('Y', 'N')),
                lit("store_and_fwd_flag inválido")
            ).otherwise(lit("")),
            when(
                col("total_amount").isNull() | (col("total_amount") <= 0),
                lit("total_amount inválido")
            ).otherwise(lit("")),
            # Verifica se o valor total pago é igual a soma das parcelas
            when(
                round(col("total_amount"), 2) != round(
                    col("fare_amount")
                    + col("extra")
                    + col("mta_tax")
                    + col("tip_amount")
                    + col("tolls_amount")
                    + col("improvement_surcharge")
                    + col("airport_fee")
                    + col("congestion_surcharge"), 2
                ),
                lit("total_amount não corresponde à soma das taxas")
            ).otherwise(lit("")),
            # Verifica se houve deslocamento através da distância percorrida
            when(
                col("trip_distance").isNull() | (col("trip_distance") <= 0),
                lit("trip_distance inválido")
            ).otherwise(lit("")),
            # Verifica se os vendorId são válidos
            when(
                col("vendorId").isNull() | (~col("vendorId").isin(vendors_validos)),
                lit("vendorId inválido")
            ).otherwise(lit(""))
        )
    )
)
# -------------------------------------------------------------------
# 2) Filtra apenas registros com algum alerta gerado
# -------------------------------------------------------------------
df_com_problemas = df_validacao.filter(col("alerta_validacao") != "")
# -------------------------------------------------------------------
# 3) Exibição para auditoria (pode trocar por write/saveAsTable)
# -------------------------------------------------------------------
df_com_problemas.display()


# 📄 Criação da View `BronzeYellowMes03`
Este script cria ou substitui a tabela `BronzeYellowMes03` no catálogo `ifood_case` e schema `nytaxi` após a validação dos dados de acordo com as regras especificadas pelo sistema de taxi de NYC e concepção de métricas auxiliares para análises posteriores.

Esta tabela será unficada com as demais tabelas dos outros meses para criar a tabela SilverYellow.

---



In [0]:
%sql

USE CATALOG ifood_case;
USE SCHEMA  nytaxi;

CREATE OR REPLACE TABLE BronzeYellowMes03 AS

SELECT
  *
FROM
  BronzeYellowMes03STG